In [1]:
%load_ext autoreload
%autoreload 2
from convnet import ConvDipNet
from timeDistributed import TimeDistributed
from torchinfo import summary
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch
import sys; sys.path.insert(0, '../')
from esinet.forward import create_forward_model, get_info
from esinet import Simulation
from copy import deepcopy
from CNN_LSTM.util import *
from dipoleDataset import DipoleDataset
import os
import mne
from esinet.evaluate import eval_auc, eval_nmse, eval_mse, eval_mean_localization_error

from util import solve_p

2025-11-25 15:07:20.575761: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 15:07:20.632729: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 15:07:21.593619: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# define hyperparameters
in_channels = 1
im_shape = (9,9)
n_filters = 8
kernel_size = (3,3)

# create single input ConvDipNet 
convnet: nn.Module  = ConvDipNet(in_channels, im_shape, n_filters, kernel_size)


# print model summary
summary(convnet, input_size=(32, 1, im_shape[0], im_shape[1])) # (batch_size, n_timesteps, in_channels, height, width)

Layer (type:depth-idx)                   Output Shape              Param #
ConvDipNet                               [32, 5124]                --
├─Conv2d: 1-1                            [32, 8, 9, 9]             80
├─BatchNorm2d: 1-2                       [32, 8, 9, 9]             16
├─Linear: 1-3                            [32, 512]                 332,288
├─BatchNorm1d: 1-4                       [32, 512]                 1,024
├─Linear: 1-5                            [32, 5124]                2,628,612
Total params: 2,962,020
Trainable params: 2,962,020
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 94.99
Input size (MB): 0.01
Forward/backward pass size (MB): 1.91
Params size (MB): 11.85
Estimated Total Size (MB): 13.76

In [3]:
model_weight_path = "/mnt/data/convdip/model/convdip_499.pt"
convnet.load_state_dict(torch.load(model_weight_path, weights_only=True))
convnet.eval()

ConvDipNet(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hidden_layer): Linear(in_features=648, out_features=512, bias=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (output_layer): Linear(in_features=512, out_features=5124, bias=True)
)

In [6]:
model_save_path = "/mnt/data/convdip/model/"
loss_save_path = "/mnt/data/convdip/model/convdip_loss.npy"
data_path = "/mnt/data/convdip/training_data/"
eeg_data_path = os.path.join(data_path, "eeg_data")
interp_data_path = os.path.join(data_path, "interp_data")
source_data_path = os.path.join(data_path, "source_data")
info_path = os.path.join(data_path, "info.fif")

dataset = DipoleDataset(eeg_data_path, interp_data_path, source_data_path, info_path, im_shape=im_shape)
test_size = 0.15
val_size = 0.15

test_amount, val_amount = int(dataset.__len__() * test_size), int(dataset.__len__() * val_size)

# this function will automatically randomly split your dataset but you could also implement the split yourself
gen = torch.Generator()
gen.manual_seed(0) # this is the seed we use to split the data the same way each time
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [
            (dataset.__len__() - (test_amount + val_amount)), 
            test_amount, 
            val_amount
], generator=gen)

B = 512  # batch size
train_dataloader = torch.utils.data.DataLoader(
            train_set,
            batch_size=B,
            shuffle=False,
)
val_dataloader = torch.utils.data.DataLoader(
            val_set,
            batch_size=B,
            shuffle=False,
)
test_dataloader = torch.utils.data.DataLoader(
            test_set,
            batch_size=B,
            shuffle=False,
)

In [7]:
dipole_pos = np.load(os.path.join(data_path, "dipole_pos.npy"))

idx, sample, target = val_dataloader.dataset[0]
print(idx)


24175


In [8]:
# create forward model
fs = 100
info = get_info(sfreq=fs)
fwd = create_forward_model(sampling='ico4', info=info)
leadfield = fwd['sol']['data']

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  80 | elapsed:    5.4s remaining:   18.6s
[Parallel(n_jobs=-1)]: Done  35 out of  80 | elapsed:    5.5s remaining:    7.1s
[Parallel(n_jobs=-1)]: Done  52 out of  80 | elapsed:    5.6s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  69 out of  80 | elapsed:    5.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    5.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  80 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  35 out of  80 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  52 out of  80 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  69 out of  80 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
convnet.to(device)
with torch.no_grad():
    metrics_per_sample = {}
    for idxs, batch, target in tqdm(val_dataloader, position=0, desc="batch"):
        batch = batch.to(device, dtype=torch.float)
        output = convnet(batch)
        output = output.cpu()

        for idx in tqdm(range(output.shape[0]), position=1, desc="sample"):
            data_idx = int(idxs[idx])
            target_sample = np.array(target[idx])
            output_sample = np.array(output[idx])
            
            eeg = np.load(os.path.join(data_path, f"eeg_data/sample_{data_idx}.npy"))
            max_idx = np.unravel_index(np.argmax(eeg), eeg.shape)[1] # this is the timestep with the maximum eeg value, this will be used to train
            output_sample = solve_p(output_sample, eeg[:,max_idx], leadfield)

            
            auc_close, auc_far = eval_auc(target_sample, output_sample, dipole_pos)
            sample_auc = (auc_close + auc_far)/2
            
            mle = eval_mean_localization_error(target_sample, output_sample, dipole_pos)
            mse = eval_mse(target_sample, output_sample)
            nmse = eval_nmse(target_sample, output_sample)
            metrics_per_sample[data_idx] = [sample_auc, mle, mse, nmse]

Using device: cuda


batch:   0%|          | 0/30 [08:14<?, ?it/s]


KeyboardInterrupt: 